In [1]:
import os
import sys
sys.path.append(os.path.abspath("../"))
from funes.agents.agent_types import Role, Persona, AutogenAgentType

from llm_foundation import logger

from dotenv import find_dotenv, load_dotenv

openai_api_key = os.environ["OPENAI_API_KEY"]

config_list = [{
    "model": "gpt-3.5-turbo",  # model name
    "api_key": openai_api_key  # api key
}]
llm_config = {
    "seed": 14,  # seed for caching and reproducibility
    "config_list": config_list,  # a list of OpenAI API configurations
    "temperature": 0.0,  # temperature for sampling
}

/opt/conda/envs/funes-pixi/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-08-31 00:06:05,427	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-08-31 00:06:05,873	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-08-31 00:06:06 fperez-gcloud-stupid-sailor-twift root[2765500] INFO Logger root configured


Trying to configure logger root in module llm_foundation
root # of associated handlers - 0
Logging is not configured yet. Configuring it now.
Basic logging config


In [2]:
!pwd

/home/jupyter/dev/funes/notebooks


In [3]:
francisco = Persona.from_json_file("Persona/Francisco.json")
print(francisco)

Persona: Francisco
  Role: learner
  Description: A highly motivated human that is eager to learn.
  Agent System Message: 
  Autogen Code Execution Config: {}



In [4]:
francisco.roles

francisco.save_as_json()

francisco.get_roles()

2024-08-31 00:06:07 fperez-gcloud-stupid-sailor-twift root[2765500] INFO JSON object written: Persona/Francisco.json


['learner']

In [5]:
francisco_learner = francisco.role_to_autogen_agent("learner", AutogenAgentType.UserProxyAgent, "ALWAYS", llm_config=llm_config)


# francisco_learner.generate_reply(
#     messages=[{"content": "Tell me a joke.", "role": "user"}]
# )


In [6]:
maestro = Persona.from_json_file("Persona/Maestro.json")

print(maestro)

maestro_teacher = maestro.role_to_autogen_agent("generic_teacher", AutogenAgentType.AssistantAgent, llm_config=llm_config) 

Persona: La_Parras
  Role: generic_teacher
  Description: A highly motivated teacher that is eager to offer her knowledge.
  Agent System Message: You are a highly motivated teacher. You write engaging and structured text answers using markdown format to questions you receive on given topics. You must polish your writing based on the feedback you receive and give a refined version. Only return your final work adding markdown format for clarity but without addint additional comments to the content.
  Autogen Code Execution Config: {}



In [7]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

critics = Persona.from_json_file("Persona/Critic.json")

print(critics)

science_critic = critics.role_to_autogen_agent("science_critic", AutogenAgentType.AssistantAgent, llm_config=llm_config)
style_editor = critics.role_to_autogen_agent("style_editor", AutogenAgentType.AssistantAgent, llm_config=llm_config)
ethics_reviewer = critics.role_to_autogen_agent("ethics_reviewer", AutogenAgentType.AssistantAgent, llm_config=llm_config)
meta_reviewer = critics.role_to_autogen_agent("meta_reviewer", AutogenAgentType.AssistantAgent, llm_config=llm_config)

print()

review_chats = [
    {
        "recipient": ethics_reviewer, 
        "message": reflection_message, 
        "summary_method": "reflection_with_llm",
        "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
        "max_turns": 1},
    {
        "recipient": style_editor, 
        "message": reflection_message, 
        "summary_method": "reflection_with_llm",
        "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
        "max_turns": 1},    
    {
        "recipient": meta_reviewer, 
        "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.",
        "max_turns": 1},
]


Persona: Carlos_Boyero
  Role: science_critic
  Description: You are reputable critic of scientific papers and technology.
  Agent System Message: You are a scientific papers and technology critic. You review the work of a writer and provide constructive feedback to help improve the quality of the content.
  Autogen Code Execution Config: {}
  Role: style_editor
  Description: You are reputable critic and editor specialized in text style.
  Agent System Message: As an expert critic and editor, you review and constructively give guidelines to make the text you receive more readable and understandable. Suggest to add markdown format rules to the content parts when necessary for clarity. Begin the review by stating your role.
  Autogen Code Execution Config: {}
  Role: ethics_reviewer
  Description: You are a philosopher specialized in ethics.
  Agent System Message: You are an ethics reviewer, known for your ability to ensure that content is ethically sound and free from any potential et

In [8]:
type(science_critic)

autogen.agentchat.assistant_agent.AssistantAgent

In [9]:
science_critic.register_nested_chats(
    review_chats,
    trigger=maestro_teacher,
)

In [10]:
from autogen import initiate_chats

task = "What is cross-validation and why is it used in machine learning?"

# chats = [
#     {
#         "sender": francisco_learner,
#         "recipient": maestro_teacher,
#         "message": task,
#         "max_turns": 1,
#         "summary_method": "last_msg",
#     }
    
# ]
# chat_results = initiate_chats(chats)

In [11]:
type(science_critic)

autogen.agentchat.assistant_agent.AssistantAgent

In [12]:
res = science_critic.initiate_chat(
    recipient=maestro_teacher,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Carlos_Boyero_science_critic (to La_Parras_generic_teacher):

What is cross-validation and why is it used in machine learning?

--------------------------------------------------------------------------------
La_Parras_generic_teacher (to Carlos_Boyero_science_critic):

Cross-validation is a technique used in machine learning to assess how well a predictive model will generalize to an independent data set. It involves partitioning the dataset into subsets, training the model on a portion of the data, and then testing it on the remaining data. This process is repeated multiple times with different partitions, and the performance metrics are averaged to provide a more accurate estimate of the model's performance. Cross-validation helps to detect overfitting and provides a more reliable evaluation of the model's ability to generalize to new, unseen data.

--------------------------------------------------------------------------------

*****************************************************

2024-08-31 00:06:10 fperez-gcloud-stupid-sailor-twift httpx[2765500] INFO HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Carlos_Boyero_style_editor (to Carlos_Boyero_science_critic):

As an expert critic and editor, I will provide feedback on the content related to cross-validation and suggest improvements for readability and clarity.

1. **Content Review**:
   - The content explains the concept of cross-validation clearly and concisely. It effectively describes the purpose and process of cross-validation in machine learning.
   - The content could benefit from using markdown format to distinguish the context section provided by the ethics reviewer. This will help in separating the main content from the additional information.

2. **Suggestions for Improvement**:
   - **Markdown Formatting**: Consider using markdown format to differentiate the main content from the context section provided by the ethics reviewer. For example:
     ```markdown
     Main Content:
     Cross-validation is a technique used in machine learning to assess how well a predictive model will generalize to an independent data set...

2024-08-31 00:06:14 fperez-gcloud-stupid-sailor-twift httpx[2765500] INFO HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



********************************************************************************
Starting a new chat....

********************************************************************************
Carlos_Boyero_science_critic (to Carlos_Boyero_meta_reviewer):

Aggregrate feedback from all reviewers and give final suggestions on the writing.
Context: 
{'Reviewer': 'Carlos_Boyero_ethics_reviewer', 'Review': 'As an ethics reviewer, I would suggest the following regarding the content provided: - Ensure that any data used for cross-validation is obtained and used in compliance with data protection regulations and guidelines to protect individuals' privacy. - Clearly state the sources of the data used for cross-validation to ensure transparency and give proper credit to the original providers. - Consider discussing the potential biases in the dataset used for cross-validation and how they may impact the generalizability of the predictive model.'}
{'Reviewer': 'Carlos_Boyero_style_editor', 'Review': '

2024-08-31 00:06:16 fperez-gcloud-stupid-sailor-twift httpx[2765500] INFO HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Carlos_Boyero_meta_reviewer (to Carlos_Boyero_science_critic):

### Aggregated Review Feedback:

**Carlos_Boyero_ethics_reviewer:**
- Suggested ensuring compliance with data protection regulations.
- Recommended stating data sources for transparency.
- Advised discussing potential biases in the dataset.

**Carlos_Boyero_style_editor:**
- Praised clear explanation of cross-validation.
- Recommended using markdown format for better organization.
- Highlighted the need for differentiation between main content and context.

### Final Suggestions on the Writing:

1. **Compliance and Transparency:** Address the suggestions from the ethics reviewer by ensuring data protection compliance, stating sources transparently, and discussing dataset biases.

2. **Markdown Formatting:** Implement the style editor's advice by using markdown format to distinguish between the main content and additional context. This will enhance readability and organization.

3. **Clarity and Readability:** By incorporat

2024-08-31 00:06:18 fperez-gcloud-stupid-sailor-twift httpx[2765500] INFO HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


La_Parras_generic_teacher (to Carlos_Boyero_science_critic):

Cross-validation is a technique used in machine learning to assess how well a predictive model will generalize to an independent data set. It involves partitioning the dataset into subsets, training the model on a portion of the data, and then testing it on the remaining data. This process is repeated multiple times with different partitions, and the performance metrics are averaged to provide a more accurate estimate of the model's performance. 

Cross-validation helps to detect overfitting and provides a more reliable evaluation of the model's ability to generalize to new, unseen data.

To enhance the content further, it is important to ensure compliance with data protection regulations, transparently state data sources, and discuss potential biases in the dataset. By incorporating markdown formatting to distinguish between the main content and additional context, the explanation of cross-validation can be more organized a

In [13]:
print(res.summary)

Cross-validation is a technique used in machine learning to assess how well a predictive model will generalize to an independent data set. It involves partitioning the dataset into subsets, training the model on a portion of the data, and then testing it on the remaining data. This process is repeated multiple times with different partitions, and the performance metrics are averaged to provide a more accurate estimate of the model's performance. 

Cross-validation helps to detect overfitting and provides a more reliable evaluation of the model's ability to generalize to new, unseen data.

To enhance the content further, it is important to ensure compliance with data protection regulations, transparently state data sources, and discuss potential biases in the dataset. By incorporating markdown formatting to distinguish between the main content and additional context, the explanation of cross-validation can be more organized and visually appealing.


In [14]:
for chat_result in res:
    print(chat_result.summary)
    print("\n")

TypeError: 'ChatResult' object is not iterable

In [ ]:
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")